# Flash CircuitPython to Particle Mesh

## Install Adafruit Bootloader (only once)

* Instructions:
  * [Particle specific](https://learn.adafruit.com/circuitpython-on-the-nrf52/build-flash-particle)
  * [General](https://learn.adafruit.com/circuitpython-on-the-nrf52/overview)
  
* [Download Bootloader](https://github.com/adafruit/Adafruit_nRF52_Bootloader/releases)
 
* Flash: Connect Particle via JLink and power the Particle device, then:
   * `nrfjprog --program particle_argon*.hex --chiperase -f nrf52 --reset`

## Install CircuitPython

* [Download CircuitPython Binary](https://github.com/adafruit/circuitpython/releases)
* Connect board, copy to `ARGONBOOT` folder
  * It will disappear, replaced by `CIRCUITPY`


## Install MicroPython on ESP32

* [Instructions (look for Argon)](https://learn.adafruit.com/adding-a-wifi-co-processor-to-circuitpython-esp8266-esp32/firmware-upload)
* Download ESP32 firmware for particle_argon (search above page)
* Copy [miniesptool.py](https://github.com/adafruit/Adafruit_CircuitPython_miniesptool) to board
* Copy firmware to ~/mcu/esp32: 
  * `github/espconnect/copy_firmware_to_mcu_esp32.sh`

In [1]:
%rsync
from adafruit_miniesptool import miniesptool
print(dir(miniesptool))

copy   'esp32_firmware/firmware.bin'
['__class__', '__init__', '__module__', '__name__', '__qualname__', 'baudrate', 'reset', 'sync', 'FLASH_WRITE_SIZE', 'FLASH_SECTOR_SIZE', 'ESP_ROM_BAUD', 'debug', 'md5', 'mac_addr', 'chip_type', 'chip_name', '_read_efuses', 'get_erase_size', 'flash_begin', 'check_command', 'send_command', 'get_response', 'read_r_r', EPL; CTRL-B to exit
nRF52840
>>> 
1a4769-dirty on 2019-0gister', 'flash_block', 'flash_file', '_sync', 'checksum', 'slip_encode']


### Copy firmware to ESP32 with `miniesptool_esp32argon.py`:
* __Note:__ this is VERY slow (~ 6 minutes)
  * Use OTA for future uploads!
* __TODO__: check `md5` signature

In [1]:
%softreset
%rsync

# miniesptool_esp32argon.py

import time
import board
import busio

from digitalio import DigitalInOut, Direction
from adafruit_miniesptool import miniesptool

baudrate = 115200
baudrate = 921600

print("Flash ESP32 on Particle Argon")

uart = busio.UART(board.ESP_RX, board.ESP_TX, baudrate=baudrate, timeout=1)

resetpin = DigitalInOut(board.ESP_WIFI_EN)
gpio0pin = DigitalInOut(board.ESP_BOOT_MODE)
esp_cts = DigitalInOut(board.ESP_CTS)
esp_cts.direction = Direction.OUTPUT
esp_cts.value = False

esptool = miniesptool(uart, gpio0pin, resetpin, flashsize=4*1024*1024)
esptool.debug = False

esptool.sync()
print("Synced")
print("Found:", esptool.chip_name)
if esptool.chip_name != "ESP32":
    raise RuntimeError("This example is for ESP32 only")
esptool.baudrate = baudrate
print("MAC ADDR: ", [hex(i) for i in esptool.mac_addr])

# 0x1000  ~/.platformio/packages/framework-arduinoespressif32@2.10001.190111/tools/sdk/bin/bootloader_dio_40m.bin 
# 0x8000  ~/Dropbox/Server/shared/projects/github/espconnect/espconnect/.pioenvs/espconnect/partitions.bin 
# 0xe000  ~/.platformio/packages/framework-arduinoespressif32@2.10001.190111/tools/partitions/boot_app0.bin 
# 0x10000 ~/Dropbox/Server/shared/projects/github/espconnect/espconnect/.pioenvs/espconnect/firmware.bin

esptool.flash_file("/esp32_firmware/bootloader_dio_40m.bin", 0x1000)
esptool.flash_file("/esp32_firmware/partitions.bin",         0x8000)
esptool.flash_file("/esp32_firmware/boot_app0.bin",          0xe000)
esptool.flash_file("/esp32_firmware/firmware.bin",          0x10000)

esptool.reset()
time.sleep(0.5)

print("DONE")

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!      softreset      !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



Directories match
Flash ESP32 on Particle Argon
Resetting
Synced
Found: ESP32
MAC ADDR:  ['0x30', '0xae', '0xa4', '0xd1', '0x73', '0x98']

Writing /esp32_firmware/bootloader_dio_40m.bin w/filesize: 16176
Erase size 16176, num_blocks 16, size 1024, offset 0x1000
Took 0.10s to erase 16 flash blocks
Writing at 0x00001000... (6 %)Writing at 0x00001400... (12 %)Writing at 0x00001800... (18 %)Writing at 0x00001c00... (25 %)Writing at 0x00002000... (31 %)Writing at 0x00002400... (37 %)Writing at 0x00002800... (43 %)Writing at 0x00002c00... (50 %)Writing at 0x00003000... (56 %)Writing at 0x00003400... (62 %)Writing at 0x00003800... (68 %)Writing at 0x00003c00... (75 %)Writing at 0x00004000... (81 %)Writing at 0x00004400... (87 %)Writing at 0x00004800... (93 %)Writing at 0x00004c00... (100 %)Took 2.69s to write 16176 bytes

Writing /esp32_firmware/partitions.bin w/filesize: 3072
Erase size 3072, num_blocks 3, size 1024, offset 0x8000
Took 0.04s to erase 3 flash blocks
Writing at 0x00008000... 

# TODO

check MD5 signature, e.g.

```python
>>> import hashlib
>>> hashlib.md5(open('filename.exe','rb').read()).hexdigest()
'd41d8cd98f00b204e9800998ecf8427e'
```